# Imports

In [3]:
import pandas as pd
from Models.scraping_models import*

# Extração e Limpeza

## DF de Cotação

In [4]:
# Fonte dos dados
fiis_data = 'https://www.clubefii.com.br/fundos_imobiliarios_ranking/2024#'
df = scraping_request(fiis_data)

# Estruturação dos dados
df[['Valor da Ação', 'Variação', 'Data', 'Hora', 'Resto']] = df['COTAÇÃO ATUAL'].str.split(' ', n=4, expand=True)
excl = ['VARIAÇÃO COTA  EM 2024', 'VARIAÇÃO COTA + PROVENTOS  EM 2024', 'Resto', 'COTAÇÃO ATUAL', 'UPSIDE/DOWNSIDE PARA ATINGIR VALOR PATRIMONIAL', 'FEED', 'Hora', 'YIELD  3 MESES', 'YIELD  6 MESES']

# Colunas indesejadas
df = df.drop(columns=excl)
df = df[['CÓDIGO', 'NOME', 'Valor da Ação', 'Variação', 'P/VPA', 'Data', 'YIELD  1 MES', 'YIELD  12 MESES']]

# Conversão de colunas para Float, para observar o percentual.
float_columns = ['Valor da Ação', 'Variação', 'YIELD  1 MES', 'YIELD  12 MESES']
df = float_transformer(df, float_columns)

# Estruturação dos dados 
df['DI Mensal'] = (df['Valor da Ação'] * (df['YIELD  1 MES'] / 100)).astype(float)
df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
df['Data'] = df['Data'].dt.strftime('%d/%m/%Y')
df = df.drop(columns='P/VPA')

## DF Classificadora

In [5]:
# Local coletado dos classificadores fees
fiis_classification = 'https://www.clubefii.com.br/fundo_imobiliario_lista#'

# Limpeza e estruturação dos dados (CLASSIFICADOR)
df_classification = scraping_request(fiis_classification)

# Colunas indesejadas
cols_to_exclude = ['VALOR COTA', 'RELATÓRIOS DE ANÁLISE', 'FEED', 'VALOR IPO','DATA IPO', 'NOME']
df_classification  = df_classification.drop(columns= cols_to_exclude)

## DF Consolidado

In [6]:
hist_name = 'Tables/FIIS_Consol.xlsx'
hist = pd.read_excel(hist_name)

In [7]:
# Classificando e concatenando os novos dados + Hist
consol = pd.merge(df_classification, df, on = 'CÓDIGO')
consol = pd.concat([hist, consol], ignore_index=False).drop_duplicates()

In [8]:
# Salvando o historico
consol.to_excel(hist_name, index= False)